In [31]:
import torch.optim
import torchvision
from torch import nn
from torch.utils.data import DataLoader, Subset
from torch.utils.tensorboard import SummaryWriter
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential
import time
from PIL import Image
import numpy as np

In [3]:
dataset_path = r'E:\file\python\MyCode\dataset\51.datasets'
train_data = torchvision.datasets.CelebA(root=dataset_path, split='train',target_type='attr',transform=torchvision.transforms.ToTensor(), download=False)
test_data = torchvision.datasets.CelebA(root=dataset_path, split='test',target_type='attr',transform=torchvision.transforms.ToTensor(), download=False)

train_loader = DataLoader(dataset=train_data, batch_size=100, shuffle=False)
test_loader = DataLoader(dataset=test_data,  batch_size=100, shuffle=False)

In [4]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.feature_process = Sequential(Conv2d(3, 30, kernel_size=50, stride=1, padding=0),
                                          MaxPool2d(2),
                                          Flatten(),
                                          Linear(30*84 * 64, 40)
                                          )



    def forward(self, x):
        x = self.feature_process(x)
        return x

cuda_available = torch.cuda.is_available()  # GPU是否可用
device = torch.device('cuda' if cuda_available else 'cpu')

demo = Model()
demo.to(device)  # 创建模型

print(train_data[0][1].size())

torch.Size([40])


In [5]:
loss_fn = nn.BCEWithLogitsLoss()  # 损失函数
loss_fn.to(device)
# 优化器
learn_rate = 1e-2
optim = torch.optim.SGD(demo.parameters(), lr=learn_rate)
# 训练次数
epochs = 10

# 画板 tensorboard --logdir='.\\example\\data\\40.nn_Conv2d' --port=6012
write = SummaryWriter('..\\data\\51.base_celeba')

In [20]:
test_data[0][0].size()

torch.Size([3, 218, 178])

In [7]:
print(len(train_loader))

1628


## 训练

In [8]:
total_train_step = 0
for epoch in range(1, 1 + epochs):
    print(f'第{epoch}轮训练'.center(50, '-'))
    epochs_train_loss = 0  # 批次训练集总损失
    epochs_test_loss = 0  # 批次训练集总损失
    for data in train_loader:
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.float()
        labels = labels.to(device)
        # 卷积层 - 处理特征
        outputs = demo(imgs)
        # 损失函数层 - 计算损失
        loss = loss_fn(outputs, labels)
        # 优化器 - 算法优化参数
        optim.zero_grad()
        loss.backward()
        optim.step()
        # 累加损失
        epochs_train_loss += loss.item()
        total_train_step += 1

        if total_train_step % 100 == 0:
            write.add_scalar('train_loss', loss.item(), total_train_step)
    print(f'第{epoch}轮训练完成，总损失{epochs_train_loss}')

    torch.save(demo.state_dict(), f'..\\data\\51.base_celeba\\CelebA_module_epoch{epoch}')

    right_num = 0
    with torch.no_grad():  # 测试的时候进入没有梯度计算模式，节省内存消耗并加快计算速度
        for data in test_loader:
            imgs, labels = data
            imgs = imgs.to(device)
            labels = labels.to(device)
            labels = labels.float()
            outputs = demo(imgs)
            # 计算测试集损失
            loss = loss_fn(outputs, labels)
            epochs_test_loss += loss.item()
            # # 测试集预测值
            # predict = outputs.argmax(dim=1)
            # # 预测正确的个数 累加
            # right_num += (predict == labels).sum()
    # right_rate = right_num / len(test_data)
    print(f'第{epoch}批次测试，损失{epochs_test_loss}')
    write.add_scalar('test_loss', epochs_test_loss, epochs)


----------------------第1轮训练-----------------------
第1轮训练完成，总损失553.0387430489063
第1批次测试，损失62.11529919505119
----------------------第2轮训练-----------------------
第2轮训练完成，总损失489.24438628554344
第2批次测试，损失59.47844064235687
----------------------第3轮训练-----------------------
第3轮训练完成，总损失473.32886785268784
第3批次测试，损失58.23290482163429
----------------------第4轮训练-----------------------
第4轮训练完成，总损失464.36201271414757
第4批次测试，损失57.439523458480835
----------------------第5轮训练-----------------------
第5轮训练完成，总损失458.16298604011536
第5批次测试，损失56.857498437166214
----------------------第6轮训练-----------------------
第6轮训练完成，总损失453.39119654893875
第6批次测试，损失56.4019401371479
----------------------第7轮训练-----------------------
第7轮训练完成，总损失449.4887733012438
第7批次测试，损失56.03125524520874
----------------------第8轮训练-----------------------
第8轮训练完成，总损失446.1728663742542
第8批次测试，损失55.721602857112885
----------------------第9轮训练-----------------------
第9轮训练完成，总损失443.28298979997635
第9批次测试，损失55.457974433898926
----------------------第10轮训练

In [76]:
chinese_name = """
5_o_Clock_Shadow：刚长出的双颊胡须
Arched_Eyebrows：柳叶眉
Attractive：吸引人的
Bags_Under_Eyes：眼袋
Bald：秃头
Bangs：刘海
Big_Lips：大嘴唇
Big_Nose：大鼻子
Black_Hair：黑发
Blond_Hair：金发
Blurry：模糊的
Brown_Hair：棕发
Bushy_Eyebrows：浓眉
Chubby：圆胖的
Double_Chin：双下巴
Eyeglasses：眼镜
Goatee：山羊胡子
Gray_Hair：灰发或白发
Heavy_Makeup：浓妆
High_Cheekbones：高颧骨
Male：男性
Mouth_Slightly_Open：微微张开嘴巴
Mustache：胡子，髭
Narrow_Eyes：细长的眼睛
No_Beard：无胡子
Oval_Face：椭圆形的脸
Pale_Skin：苍白的皮肤
Pointy_Nose：尖鼻子
Receding_Hairline：发际线后移
Rosy_Cheeks：红润的双颊
Sideburns：连鬓胡子
Smiling：微笑
Straight_Hair：直发
Wavy_Hair：卷发
Wearing_Earrings：戴耳环
Wearing_Hat：戴帽子
Wearing_Lipstick：涂口红
Wearing_Necklace：戴项链
Wearing_Necktie：戴领带
Young：年轻
"""
chinese_name_ls = chinese_name.replace('\n',',').strip(',').split(',')
chinese_name_dict = {i.split('：')[0]:i.split('：')[1] for i in chinese_name_ls}
attr_names = np.array(train_data.attr_names)
img_path = '..\\..\\dataset\\51.datasets\\test\\7.png'
img = Image.open(img_path)
img = img.convert("RGB")
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((218, 178)),torchvision.transforms.ToTensor()])
img = transform(img)
# demo.eval()
img = torch.reshape(img, (1,  3, 218, 178))
with torch.no_grad():
    img = img.to(device)
    output = demo(img)
    output = torch.sigmoid(output)
    output = torch.round(output)
    output = output.cpu()
    output = output.int()
    output = np.array(output)[0]
    output_english = attr_names[np.where(output == 1)[0]]
    for i in output_english:
        print(chinese_name_dict.get(i))

大鼻子
金发
双下巴
无胡子
发际线后移
微笑
涂口红
